In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

In [2]:
data_frame=pd.read_csv('data.csv')

In [3]:
data_frame.head()

ID Case Number                    Date                      Block  \
0   5741943    HN549294  08/25/2007 09:22:18 AM         074XX N ROGERS AVE   
1     25953    JE240540  05/24/2021 03:06:00 PM        020XX N LARAMIE AVE   
2     26038    JE279849  06/26/2021 09:24:00 AM      062XX N MC CORMICK RD   
3  13279676    JG507211  11/09/2023 07:30:00 AM           019XX W BYRON ST   
4  13274752    JG501049  11/12/2023 07:59:00 AM  086XX S COTTAGE GROVE AVE   

   IUCR Primary Type                                        Description  \
0  0560      ASSAULT                                             SIMPLE   
1  0110     HOMICIDE                                FIRST DEGREE MURDER   
2  0110     HOMICIDE                                FIRST DEGREE MURDER   
3  0620     BURGLARY                                     UNLAWFUL ENTRY   
4  0454      BATTERY  AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...   

  Location Description  Arrest  Domestic  Beat  District  Ward  \
0                OTHER   False     False  2422      24.0  49.0   
1               STREET    True     False  2515      25.0  36.0   
2          PARKING LOT    True     False  1711      17.0  50.0   
3            APARTMENT   False     False  1922      19.0  47.0   
4   SMALL RETAIL STORE    True     False   632       6.0   6.0   

   Community Area FBI Code  X Coordinate  Y Coordinate  Year  \
0             1.0      08A           NaN           NaN  2007   
1            19.0      01A     1141387.0     1913179.0  2021   
2            13.0      01A     1152781.0     1941458.0  2021   
3             5.0       05     1162518.0     1925906.0  2023   
4            44.0      08B     1183071.0     1847869.0  2023   

               Updated On   Latitude  Longitude                       Location  
0  08/17/2015 03:03:40 PM        NaN        NaN                            NaN  
1  11/18/2023 03:39:49 PM  41.917838 -87.755969  (41.917838056, -87.755968972)  
2  11/18/2023 03:39:49 PM  41.995219 -87.713355  (41.995219444, -87.713354912)  
3  11/18/2023 03:39:49 PM  41.952345 -87.677975  (41.952345086, -87.677975059)  
4  12/09/2023 03:41:24 PM  41.737751 -87.604856  (41.737750767, -87.604855911)

In [4]:
#getting the null value count in the dataset
def null_count_gen(data_frame):
    null_count=data_frame.isnull().sum()
    null_per=data_frame.isnull().sum().values/len(data_frame)*100
    null_count_df=pd.DataFrame({'Null count':null_count,
                                'Null percentage(%)':null_per,
                               'dtypes':data_frame.dtypes},index=data_frame.columns)
    return null_count_df

In [5]:
null_count_gen(data_frame).sort_values('Null percentage(%)')

Null count  Null percentage(%)   dtypes
ID                             0            0.000000    int64
Updated On                     0            0.000000   object
Year                           0            0.000000    int64
FBI Code                       0            0.000000   object
Domestic                       0            0.000000     bool
Arrest                         0            0.000000     bool
Beat                           0            0.000000    int64
Primary Type                   0            0.000000   object
IUCR                           0            0.000000   object
Block                          0            0.000000   object
Date                           0            0.000000   object
Case Number                    0            0.000000   object
Description                    0            0.000000   object
District                      47            0.000589  float64
Location Description       12364            0.154904   object
Longitude                  87847            1.100604  float64
X Coordinate               87847            1.100604  float64
Y Coordinate               87847            1.100604  float64
Latitude                   87847            1.100604  float64
Location                   87847            1.100604   object
Community Area            613479            7.686059  float64
Ward                      614854            7.703286  float64

In [6]:
chicago_wards = {
    "1st Ward": ["Wicker Park", "Bucktown", "Ukrainian Village"],
    "2nd Ward": ["Wicker Park", "Ukrainian Village", "Lincoln Park", "Old Town"],
    "3rd Ward": ["Bronzeville", "South Loop", "Near South Side"],
    "4th Ward": ["Bronzeville", "Hyde Park", "South Shore"],
    "5th Ward": ["Hyde Park", "Woodlawn", "South Shore"],
    "6th Ward": ["Chatham", "Greater Grand Crossing", "Park Manor"],
    "7th Ward": ["South Shore", "South Chicago", "Calumet Heights"],
    "8th Ward": ["South Chicago", "Calumet Heights", "Pullman"],
    "9th Ward": ["Roseland", "Pullman", "South Deering"],
    "10th Ward": ["Hegewisch", "East Side", "South Chicago"],
    "11th Ward": ["Bridgeport", "McKinley Park", "Brighton Park"],
    "12th Ward": ["McKinley Park", "Brighton Park", "Little Village"],
    "13th Ward": ["Clearing", "Garfield Ridge", "West Lawn"],
    "14th Ward": ["Archer Heights", "Gage Park", "West Elsdon"],
    "15th Ward": ["West Englewood", "Gage Park", "Chicago Lawn"],
    "16th Ward": ["West Englewood", "Chicago Lawn", "Marquette Park"],
    "17th Ward": ["Englewood", "Auburn Gresham", "Roseland"],
    "18th Ward": ["Austin", "Montclare", "Belmont Cragin"],
    "19th Ward": ["Beverly", "Mount Greenwood", "Morgan Park"],
    "20th Ward": ["Auburn Gresham", "Englewood", "Washington Park"],
    "21st Ward": ["Auburn Gresham", "Englewood", "West Englewood"],
    "22nd Ward": ["Little Village", "Lawndale", "Brighton Park"],
    "23rd Ward": ["Clearing", "Garfield Ridge", "Archer Heights"],
    "24th Ward": ["North Lawndale", "East Garfield Park", "Little Village"],
    "25th Ward": ["Pilsen", "Little Village", "University Village"],
    "26th Ward": ["Humboldt Park", "Logan Square", "Hermosa"],
    "27th Ward": ["Humboldt Park", "Logan Square", "Avondale"],
    "28th Ward": ["East Garfield Park", "Humboldt Park", "West Garfield Park"],
    "29th Ward": ["Austin", "Galewood", "Montclare"],
    "30th Ward": ["Belmont Cragin", "Portage Park", "Hermosa"],
    "31st Ward": ["Belmont Cragin", "Hermosa", "Logan Square"],
    "32nd Ward": ["Bucktown", "Logan Square", "Avondale"],
    "33rd Ward": ["Albany Park", "Irving Park", "Avondale"],
    "34th Ward": ["Albany Park", "Irving Park", "Portage Park"],
    "35th Ward": ["Logan Square", "Avondale", "Irving Park"],
    "36th Ward": ["Portage Park", "Belmont Cragin", "Hermosa"],
    "37th Ward": ["Austin", "Humboldt Park", "Belmont Cragin"],
    "38th Ward": ["Portage Park", "Dunning", "Belmont Cragin"],
    "39th Ward": ["Sauganash", "Edgebrook", "North Park"],
    "40th Ward": ["Edgewater", "West Ridge", "Rogers Park"],
    "41st Ward": ["Edgebrook", "Norwood Park", "O'Hare"],
    "42nd Ward": ["River North", "Gold Coast", "Near North Side"],
    "43rd Ward": ["Lincoln Park", "Old Town", "Lakeview"],
    "44th Ward": ["Lakeview", "Uptown", "Buena Park"],
    "45th Ward": ["Jefferson Park", "Portage Park", "Mayfair"],
    "46th Ward": ["Uptown", "Edgewater", "Buena Park"],
    "47th Ward": ["Lincoln Square", "Ravenswood", "North Center"],
    "48th Ward": ["Edgewater", "Uptown", "Andersonville"],
    "49th Ward": ["Rogers Park", "Edgewater", "West Ridge"],
    "50th Ward": ["West Ridge", "Peterson Park", "West Rogers Park"]
}


In [7]:
data_frame.head()

ID Case Number                    Date                      Block  \
0   5741943    HN549294  08/25/2007 09:22:18 AM         074XX N ROGERS AVE   
1     25953    JE240540  05/24/2021 03:06:00 PM        020XX N LARAMIE AVE   
2     26038    JE279849  06/26/2021 09:24:00 AM      062XX N MC CORMICK RD   
3  13279676    JG507211  11/09/2023 07:30:00 AM           019XX W BYRON ST   
4  13274752    JG501049  11/12/2023 07:59:00 AM  086XX S COTTAGE GROVE AVE   

   IUCR Primary Type                                        Description  \
0  0560      ASSAULT                                             SIMPLE   
1  0110     HOMICIDE                                FIRST DEGREE MURDER   
2  0110     HOMICIDE                                FIRST DEGREE MURDER   
3  0620     BURGLARY                                     UNLAWFUL ENTRY   
4  0454      BATTERY  AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...   

  Location Description  Arrest  Domestic  Beat  District  Ward  \
0                OTHER   False     False  2422      24.0  49.0   
1               STREET    True     False  2515      25.0  36.0   
2          PARKING LOT    True     False  1711      17.0  50.0   
3            APARTMENT   False     False  1922      19.0  47.0   
4   SMALL RETAIL STORE    True     False   632       6.0   6.0   

   Community Area FBI Code  X Coordinate  Y Coordinate  Year  \
0             1.0      08A           NaN           NaN  2007   
1            19.0      01A     1141387.0     1913179.0  2021   
2            13.0      01A     1152781.0     1941458.0  2021   
3             5.0       05     1162518.0     1925906.0  2023   
4            44.0      08B     1183071.0     1847869.0  2023   

               Updated On   Latitude  Longitude                       Location  
0  08/17/2015 03:03:40 PM        NaN        NaN                            NaN  
1  11/18/2023 03:39:49 PM  41.917838 -87.755969  (41.917838056, -87.755968972)  
2  11/18/2023 03:39:49 PM  41.995219 -87.713355  (41.995219444, -87.713354912)  
3  11/18/2023 03:39:49 PM  41.952345 -87.677975  (41.952345086, -87.677975059)  
4  12/09/2023 03:41:24 PM  41.737751 -87.604856  (41.737750767, -87.604855911)

1. Most common types on crimes reported in Chicago

In [8]:
data_frame['Primary Type'].value_counts()

Primary Type
THEFT                                1685702
BATTERY                              1456320
CRIMINAL DAMAGE                       909852
NARCOTICS                             751559
ASSAULT                               524354
OTHER OFFENSE                         495238
BURGLARY                              429910
MOTOR VEHICLE THEFT                   396459
DECEPTIVE PRACTICE                    357994
ROBBERY                               301005
CRIMINAL TRESPASS                     217723
WEAPONS VIOLATION                     112804
PROSTITUTION                           69965
OFFENSE INVOLVING CHILDREN             57163
PUBLIC PEACE VIOLATION                 52966
SEX OFFENSE                            31816
CRIM SEXUAL ASSAULT                    27540
INTERFERENCE WITH PUBLIC OFFICER       18811
LIQUOR LAW VIOLATION                   15034
GAMBLING                               14631
ARSON                                  13645
HOMICIDE                               130

2. Distribution of crimes in different neighbourhoods

In [9]:
np.sort(data_frame['Community Area'].unique())

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       nan])

In [10]:
data_frame[data_frame['Community Area']==0]

ID Case Number                    Date                   Block  \
550142   1975343    HH167508  02/04/2002 07:30:00 PM    0000X S LAKEFRONT DR   
830716   2227605    HH499526  07/09/2002 10:00:00 AM  110XX S CALIFORNIA AVE   
914660   2232566    HH492414  07/05/2002 10:30:00 AM  051XX S RUTHERFORD AVE   
930623   2267024    HH546139  02/02/2002 02:00:00 AM  051XX S RUTHERFORD AVE   
950702   2502381    HH843643  12/17/2002 12:00:00 AM  051XX S RUTHERFORD AVE   
...          ...         ...                     ...                     ...   
4722628  8197932    HT431857  08/04/2011 05:00:00 PM  051XX S RUTHERFORD AVE   
4733539  8274387    HT507295  09/21/2011 12:45:00 PM  050XX N CUMBERLAND AVE   
4832803  8150814    HT384983  07/07/2011 02:00:00 PM  051XX S RUTHERFORD AVE   
4866337  8487665    HV164869  02/19/2012 06:54:00 PM  051XX S RUTHERFORD AVE   
4954565  8389450    HT622044  04/15/2008 03:00:00 PM   081XX W CATHERINE AVE   

         IUCR         Primary Type          Description  \
550142   0910  MOTOR VEHICLE THEFT           AUTOMOBILE   
830716   1320      CRIMINAL DAMAGE           TO VEHICLE   
914660   2820        OTHER OFFENSE     TELEPHONE THREAT   
930623   0281  CRIM SEXUAL ASSAULT       NON-AGGRAVATED   
950702   0810                THEFT            OVER $500   
...       ...                  ...                  ...   
4722628  2820        OTHER OFFENSE     TELEPHONE THREAT   
4733539  0460              BATTERY               SIMPLE   
4832803  0560              ASSAULT               SIMPLE   
4866337  1305      CRIMINAL DAMAGE  CRIMINAL DEFACEMENT   
4954565  0890                THEFT        FROM BUILDING   

                  Location Description  Arrest  Domestic  Beat  District  \
550142                          STREET   False     False   124       1.0   
830716                          STREET   False     False  2212      22.0   
914660                       RESIDENCE   False      True   811       8.0   
930623                          STREET   False     False   811       8.0   
950702                           OTHER   False     False   811       8.0   
...                                ...     ...       ...   ...       ...   
4722628                      RESIDENCE   False     False   811       8.0   
4733539                   CTA PLATFORM   False     False  1614      16.0   
4832803  RESIDENTIAL YARD (FRONT/BACK)   False      True   811       8.0   
4866337               RESIDENCE-GARAGE   False     False   811       8.0   
4954565                      RESIDENCE   False      True  1611      16.0   

         Ward  Community Area FBI Code  X Coordinate  Y Coordinate  Year  \
550142   42.0             0.0       07     1179430.0     1900237.0  2002   
830716   19.0             0.0       14     1159683.0     1831393.0  2002   
914660   23.0             0.0       26     1132251.0     1869772.0  2002   
930623   23.0             0.0       02     1132249.0     1869846.0  2002   
950702   23.0             0.0       06     1132249.0     1869846.0  2002   
...       ...             ...      ...           ...           ...   ...   
4722628  23.0             0.0       26     1132250.0     1869806.0  2011   
4733539  41.0             0.0      08B     1119317.0     1932237.0  2011   
4832803  23.0             0.0      08A     1132250.0     1869806.0  2011   
4866337  23.0             0.0       14     1132249.0     1869846.0  2012   
4954565   NaN             0.0       06           NaN           NaN  2008   

                     Updated On   Latitude  Longitude  \
550142   02/28/2018 03:56:25 PM  41.881537 -87.616595   
830716   02/28/2018 03:56:25 PM  41.693049 -87.690994   
914660   02/28/2018 03:56:25 PM  41.798886 -87.790544   
930623   02/10/2018 03:50:01 PM  41.799090 -87.790549   
950702   02/28/2018 03:56:25 PM  41.799090 -87.790549   
...                         ...        ...        ...   
4722628  02/10/2018 03:50:01 PM  41.798980 -87.790547   
4733539  02/10/2018 03:50:01 PM  41.970515 -87.836651   
48328

In [11]:
community_areas = {
    1: 'Rogers Park',
    2: 'West Ridge',
    3: 'Uptown',
    4: 'Lincoln Square',
    5: 'North Center',
    6: 'Lakeview',
    7: 'Lincoln Park',
    8: 'Near North Side',
    9: 'Edison Park',
    10: 'Norwood Park',
    11: 'Jefferson Park',
    12: 'Forest Glen',
    13: 'North Park',
    14: 'Albany Park',
    15: 'Portage Park',
    16: 'Irving Park',
    17: 'Dunning',
    18: 'Montclare',
    19: 'Belmont Cragin',
    20: 'Hermosa',
    21: 'Avondale',
    22: 'Logan Square',
    23: 'Humboldt Park',
    24: 'West Town',
    25: 'Austin',
    26: 'West Garfield Park',
    27: 'East Garfield Park',
    28: 'Near West Side',
    29: 'North Lawndale',
    30: 'South Lawndale',
    31: 'Lower West Side',
    32: 'Loop',
    33: 'Near South Side',
    34: 'Armour Square',
    35: 'Douglas',
    36: 'Oakland',
    37: 'Fuller Park',
    38: 'Grand Boulevard',
    39: 'Kenwood',
    40: 'Washington Park',
    41: 'Hyde Park',
    42: 'Woodlawn',
    43: 'South Shore',
    44: 'Chatham',
    45: 'Avalon Park',
    46: 'South Chicago',
    47: 'Burnside',
    48: 'Calumet Heights',
    49: 'Roseland',
    50: 'Pullman',
    51: 'South Deering',
    52: 'East Side',
    53: 'West Pullman',
    54: 'Riverdale',
    55: 'Hegewisch',
    56: 'Garfield Ridge',
    57: 'Archer Heights',
    58: 'Brighton Park',
    59: 'McKinley Park',
    60: 'Bridgeport',
    61: 'New City',
    62: 'West Elsdon',
    63: 'Gage Park',
    64: 'Clearing',
    65: 'West Lawn',
    66: 'Chicago Lawn',
    67: 'West Englewood',
    68: 'Englewood',
    69: 'Greater Grand Crossing',
    70: 'Ashburn',
    71: 'Auburn Gresham',
    72: 'Beverly',
    73: 'Washington Heights',
    74: 'Mount Greenwood',
    75: 'Morgan Park',
    76: "O'Hare",
    77: 'Edgewater'
}

In [12]:
chicago_beat_names = {
    2422: 'Near West Side',
    2515: 'Grand Boulevard',
    1711: 'Rogers Park',
    1922: 'West Ridge',
    632: 'Woodlawn',
    512: 'Calumet',
    122: 'Near North',
    1225: 'Near West',
    333: 'Near South',
    1732: 'West Pullman',
    822: 'Chatham',
    835: 'Roseland',
    731: 'Chicago Lawn',
    2222: 'Belmont Cragin',
    735: 'West Pullman',
    915: 'Washington Heights',
    1124: 'South Lawndale',
    1122: 'Humboldt Park',
    2513: 'Grand Boulevard',
    1925: 'West Ridge',
    1111: 'Near North',
    432: 'Grand Boulevard',
    1211: 'Near West Side',
    2232: 'Rogers Park',
    631: 'Englewood',
    411: 'Douglas',
    923: 'Washington Heights',
    423: 'Grand Boulevard',
    724: 'Chicago Lawn',
    324: 'Near North Side',
    1621: 'Austin',
    733: 'Gage Park',
    1031: 'Morgan Park',
    2521: 'Grand Boulevard',
    2514: 'Douglas',
    2522: 'Grand Boulevard',
    1433: 'West Pullman',
    531: 'Roseland',
    224: 'West Ridge',
    232: 'West Ridge',
    1431: 'Morgan Park',
    1412: 'West Pullman',
    1421: 'Morgan Park',
    1432: 'West Pullman',
    1424: 'Morgan Park',
    1013: 'West Pullman',
    1712: 'Rogers Park',
    1422: 'Morgan Park',
    833: 'Grand Boulevard',
    2512: 'Grand Boulevard',
    2532: 'Grand Boulevard',
    2524: 'Grand Boulevard',
    621: 'Englewood',
    2533: 'Grand Boulevard',
    831: 'Chatham',
    1622: 'Austin',
    1634: 'West Ridge',
    1021: 'West Pullman',
    1131: 'Austin',
    612: 'Englewood',
    1713: 'Rogers Park',
    815: 'Chatham',
    533: 'Roseland',
    421: 'Douglas',
    722: 'Chicago Lawn',
    321: 'Near North Side',
    2024: 'West Pullman',
    821: 'Chatham',
    1233: 'Rogers Park',
    1722: 'Rogers Park',
    1823: 'West Ridge',
    323: 'Near North Side',
    511: 'Calumet',
    1423: 'West Pullman',
    925: 'Washington Heights',
    613: 'Englewood',
    1235: 'Rogers Park',
    133: 'Near North',
    914: 'Washington Heights',
    1914: 'West Ridge',
    412: 'Douglas',
    1511: 'Calumet',
    2013: 'Grand Boulevard',
    222: 'Near West Side',
    1731: 'West Pullman',
    2423: 'Near West Side',
    1611: 'Austin',
    322: 'Near North Side',
    1533: 'West Pullman',
    1115: 'Near North',
    434: 'Grand Boulevard',
    2534: 'Grand Boulevard',
    1214: 'Near West Side',
    2032: 'West Pullman',
    132: 'Near North',
    825: 'Chatham',
    1135: 'Austin',
    823: 'Chatham',
    1523: 'West Pullman',
    215: 'West Ridge',
    633: 'Englewood',
    2234: 'Rogers Park',
    611: 'Englewood',
    2031: 'West Pullman',
    1623: 'Austin',
    712: 'Chicago Lawn',
    313: 'Near North Side',
    1913: 'West Ridge',
    433: 'Grand Boulevard',
    1524: 'West Pullman',
    935: 'Washington Heights',
    1133: 'Austin',
    1531: 'West Pullman',
    1911: 'West Ridge',
    1012: 'West Pullman',
    1915: 'West Ridge',
    734: 'Chicago Lawn',
    522: 'Calumet',
    124: 'Near North',
    211: 'Near North Side',
    331: 'Near North Side',
    2413: 'Near West Side',
    1134: 'Austin',
    1650: 'Austin',
    431: 'Douglas',
    221: 'Near West Side',
    1733: 'West Pullman',
    1821: 'West Ridge',
    1921: 'West Ridge',
    2221: 'Near West Side',
    634: 'Englewood',
    414: 'Douglas',
    2432: 'Near West Side',
    114: 'Near North',
    2433: 'Near West Side',
    1112: 'Near North',
    1832: 'West Ridge',
    834: 'Grand Boulevard',
    832: 'Grand Boulevard',
    422: 'Douglas',
    2211: 'Near West Side',
    1824: 'West Ridge',
    2223: 'Near West Side',
    624: 'Englewood',
    1931: 'West Ridge',
    2525: 'Grand Boulevard',
    1113: 'Near North',
    1833: 'West Ridge',
    1631: 'Austin',
    1121: 'Humboldt Park',
    1655: 'Austin',
    726: 'Chicago Lawn',
    123: 'Near North',
    934: 'Washington Heights',
    2411: 'Near West Side',
    2431: 'Near West Side',
    1114: 'Near North',
    614: 'Englewood',
    1213: 'Near West Side',
    523: 'Calumet',
    824: 'Chatham',
    312: 'Near North Side',
    1414: 'West Pullman',
    225: 'West Ridge',
    524: 'Calumet',
    732: 'Gage Park',
    1123: 'Humboldt Park',
    2011: 'Grand Boulevard',
    1811: 'West Ridge',
    932: 'Washington Heights',
    1234: 'Rogers Park',
    1224: 'Near North',
    1724: 'Rogers Park',
    532: 'Roseland',
    1831: 'West Ridge',
    1814: 'West Ridge',
    214: 'West Ridge',
    813: 'Chatham',
    1014: 'West Pullman',
    924: 'Washington Heights',
    1834: 'West Ridge',
    1232: 'Rogers Park',
    921: 'Washington Heights',
    131: 'Near North',
    931: 'Washington Heights',
    424: 'Douglas',
    1633: 'West Ridge',
    413: 'Douglas',
    1222: 'Near North',
    2012: 'Grand Boulevard',
    231: 'Near West Side',
    1011: 'West Pullman',
    2412: 'Near West Side',
    2424: 'Near West Side',
    713: 'Chicago Lawn',
}

In [13]:
beat_names = {
    '2422': 'Near West Side',
    '2515': 'Grand Boulevard',
    '1711': 'Rogers Park',
    '1922': 'West Ridge',
    '632': 'Woodlawn',
    '512': 'Calumet',
    '122': 'Near North',
    '1225': 'Near West',
    '333': 'Near South',
    '1732': 'West Pullman',
    '822': 'Chatham',
    '835': 'Roseland',
    '731': 'Chicago Lawn',
    '2222': 'Belmont Cragin',
    '735': 'West Pullman',
    '915': 'Washington Heights',
    '1124': 'South Lawndale',
    '1122': 'Humboldt Park',
    '2513': 'Grand Boulevard',
    '1925': 'West Ridge',
    '1111': 'Near North',
    '432': 'Grand Boulevard',
    '1211': 'Near West Side',
    '2232': 'Rogers Park',
    '631': 'Englewood',
    '411': 'Douglas',
    '923': 'Washington Heights',
    '423': 'Grand Boulevard',
    '724': 'Chicago Lawn',
    '324': 'Near North Side',
    '1621': 'Austin',
    '733': 'Gage Park',
    '1031': 'Morgan Park',
    '2521': 'Grand Boulevard',
    '2514': 'Douglas',
    '2522': 'Grand Boulevard',
    '1433': 'West Pullman',
    '531': 'Roseland',
    '224': 'West Ridge',
    '232': 'West Ridge',
    '1431': 'Morgan Park',
    '1412': 'West Pullman',
    '1421': 'Morgan Park',
    '1432': 'West Pullman',
    '1424': 'Morgan Park',
    '1013': 'West Pullman',
    '1712': 'Rogers Park',
    '1422': 'Morgan Park',
    '833': 'Grand Boulevard',
    '2512': 'Grand Boulevard',
    '2532': 'Grand Boulevard',
    '2524': 'Grand Boulevard',
    '621': 'Englewood',
    '2533': 'Grand Boulevard',
    '831': 'Chatham',
    '1622': 'Austin',
    '1634': 'West Ridge',
    '1021': 'West Pullman',
    '1131': 'Austin',
    '612': 'Englewood',
    '1713': 'Rogers Park',
    '815': 'Chatham',
    '533': 'Roseland',
    '421': 'Douglas',
    '722': 'Chicago Lawn',
    '321': 'Near North Side',
    '2024': 'West Pullman',
    '821': 'Chatham',
    '1233': 'Rogers Park',
    '1722': 'Rogers Park',
    '1823': 'West Ridge',
    '323': 'Near North Side',
    '511': 'Calumet',
    '1423': 'West Pullman',
    '925': 'Washington Heights',
    '613': 'Englewood',
    '1235': 'Rogers Park',
    '133': 'Near North',
    '914': 'Washington Heights',
    '1914': 'West Ridge',
    '412': 'Douglas',
    '1511': 'Calumet',
    '2013': 'Grand Boulevard',
    '222': 'Near West Side',
    '1731': 'West Pullman',
    '2423': 'Near West Side',
    '1611': 'Austin',
    '322': 'Near North Side',
    '1533': 'West Pullman',
    '1115': 'Near North',
    '434': 'Grand Boulevard',
    '2534': 'Grand Boulevard',
    '1214': 'Near West Side',
    '2032': 'West Pullman',
    '132': 'Near North',
    '825': 'Chatham',
    '1135': 'Austin',
    '823': 'Chatham',
    '1523': 'West Pullman',
    '215': 'West Ridge',
    '633': 'Englewood',
    '2234': 'Rogers Park',
    '611': 'Englewood',
    '2031': 'West Pullman',
    '1623': 'Austin',
    '712': 'Chicago Lawn',
    '313': 'Near North Side',
    '1913': 'West Ridge',
    '433': 'Grand Boulevard',
    '1524': 'West Pullman',
    '935': 'Washington Heights',
    '1133': 'Austin',
    '1531': 'West Pullman',
    '1911': 'West Ridge',
    '1012': 'West Pullman',
    '1915': 'West Ridge',
    '734': 'Chicago Lawn',
    '522': 'Calumet',
    '124': 'Near North',
    '211': 'Near North Side',
    '331': 'Near North Side',
    '2413': 'Near West Side',
    '1134': 'Austin',
    '1650': 'Austin',
    '431': 'Douglas',
    '221': 'Near West Side',
    '1733': 'West Pullman',
    '1821': 'West Ridge',
    '1921': 'West Ridge',
    '2221': 'Near West Side',
    '634': 'Englewood',
    '414': 'Douglas',
    '2432': 'Near West Side',
    '114': 'Near North',
    '2433': 'Near West Side',
    '1112': 'Near North',
    '1832': 'West Ridge',
    '834': 'Grand Boulevard',
    '832': 'Grand Boulevard',
    '422': 'Douglas',
    '2211': 'Near West Side',
    '1824': 'West Ridge',
    '2223': 'Near West Side',
    '624': 'Englewood',
    '1931': 'West Ridge',
    '2525': 'Grand Boulevard',
    '1113': 'Near North',
    '1833': 'West Ridge',
    '1631': 'Austin',
    '1121': 'Humboldt Park',
    '1655': 'Austin',
    '726': 'Chicago Lawn',
    '123': 'Near North',
    '934': 'Washington Heights',
    '2411': 'Near West Side',
    '2431': 'Near West Side',
    '1114': 'Near North',
    '614': 'Englewood',
    '1213': 'Near West Side',
    '523': 'Roseland',
    '824': 'Chatham',
    '312': 'Near North Side',
    '1414': 'West Pullman',
    '225': 'West Ridge',
    '524': 'Roseland',
    '732': 'Chicago Lawn',
    '1123': 'Humboldt Park',
    '2011': 'Grand Boulevard',
    '1811': 'West Ridge',
    '932': 'Washington Heights',
    '1234': 'Rogers Park',
    '1224': 'Near West Side',
    '1724': 'Rogers Park',
    '532': 'Roseland',
    '1831': 'West Ridge',
    '1814': 'West Ridge',
    '214': 'West Ridge',
    '813': 'Chatham',
    '1014': 'West Pullman',
    '924': 'Washington Heights',
    '1834': 'West Ridge',
    '1232': 'Rogers Park',
    '921': 'Washington Heights',
    '131': 'Near North',
    '931': 'Washington Heights',
    '424': 'Grand Boulevard',
    '1633': 'Austin',
    '413': 'Douglas',
    '1222': 'Near West Side',
    '2012': 'Grand Boulevard',
    '231': 'Near West Side',
    '1011': 'West Pullman',
    '2412': 'Near West Side',
    '2424': 'Near West Side',
    '713': 'Chicago Lawn',
    '1522': 'West Pullman',
    '2212': 'Near West Side',
    '1632': 'Austin',
    '1934': 'West Ridge',
    '623': 'Englewood',
    '1434': 'West Pullman',
    '1413': 'West Pullman',
    '2523': 'Grand Boulevard',
    '332': 'Near North',
    '1132': 'Austin',
    '2022': 'Grand Boulevard',
    '112': 'Near North',
    '1723': 'Rogers Park',
    '1651': 'Austin',
    '113': 'Near North',
    '1223': 'Near West Side',
    '1624': 'Austin',
    '1924': 'West Ridge',
    '1212': 'Near West Side',
    '814': 'Chatham',
    '334': 'Near North Side',
    '1822': 'West Ridge',
    '233': 'Near West Side',
    '111': 'Near North',
    '311': 'Near North Side',
    '1411': 'West Pullman',
    '1221': 'Near West Side',
    '513': 'Calumet',
    '234': 'Near West Side',
    '1125': 'Humboldt Park',
    '235': 'Near West Side',
    '121': 'Near West Side',
    '1532': 'West Pullman',
    '213': 'West Ridge',
    '2511': 'Grand Boulevard',
    '1512': 'Calumet',
    '1813': 'West Ridge',
    '314': 'Near North Side',
    '1923': 'West Ridge',
    '1231': 'Rogers Park',
    '1032': 'Morgan Park',
    '811': 'Chatham',
    '1912': 'West Ridge',
    '933': 'Washington Heights',
    '1215': 'Near West Side',
    '1024': 'Morgan Park',
    '922': 'Washington Heights',
    '1612': 'Austin',
    '715': 'Chicago Lawn',
    '723': 'Chicago Lawn',
    '1613': 'Austin',
    '1935': 'West Ridge',
    '1513': 'Calumet',
    '1023': 'Morgan Park',
    '622': 'Englewood',
    '913': 'Washington Heights',
    '2023': 'Grand Boulevard',
    '912': 'Washington Heights',
    '1022': 'Morgan Park',
    '725': 'Chicago Lawn',
    '1614': 'Austin',
    '1933': 'West Ridge',
    '223': 'Near West Side',
    '2535': 'Grand Boulevard',
    '1654': 'Austin',
    '1033': 'Morgan Park',
    '1812': 'West Ridge',
    '2531': 'Grand Boulevard',
    '1034': 'Morgan Park',
    '812': 'Chatham',
    '212': 'West Ridge',
    '1932': 'West Ridge',
    '714': 'Chicago Lawn',
    '711': 'Chicago Lawn',
    '911': 'Washington Heights',
    '2233': 'Rogers Park',
    '2033': 'Grand Boulevard',
    '2213': 'Near West Side',
    '1653': 'Austin',
    '1652': 'Austin',
    '1313': 'Near North',
    '2312': 'Near West Side',
    '1324': 'Near North',
    '2113': 'West Ridge',
    '1332': 'Near North',
    '2123': 'West Ridge',
    '2311': 'Near West Side',
    '2132': 'West Ridge',
    '2124': 'West Ridge',
    '2122': 'West Ridge',
    '134': 'Near North',
    '1323': 'Near North',
    '2112': 'West Ridge',
    '2332': 'Near West Side',
    '2133': 'West Ridge',
    '2333': 'Near West Side',
    '1311': 'Near North',
    '1333': 'Near North',
    '2331': 'Near West Side',
    '1312': 'Near North',
    '2323': 'Near West Side',
    '1322': 'Near North',
    '2131': 'West Ridge',
    '2324': 'Near West Side',
    '2111': 'West Ridge',
    '2313': 'Near West Side',
    '1331': 'Near North',
    '2322': 'Near West Side',
    '310': 'Near North Side',
    '430': 'Grand Boulevard'
}


In [14]:
chicago_district_names = {
    '24': 'Rogers Park',
    '25': 'Grand Central',
    '17': 'Albany Park',
    '19': 'Town Hall',
    '6': 'Gresham',
    '5': 'Calumet',
    '1': 'Central',
    '12': 'Near North',
    '3': 'Grand Crossing',
    '8': 'Chicago Lawn',
    '7': 'Englewood',
    '22': 'Morgan Park',
    '9': 'Deering',
    '11': 'Harrison',
    '4': 'South Chicago',
    '16': 'Jefferson Park',
    '10': 'Ogden',
    '14': 'Shakespeare',
    '2': 'Wentworth',
    '20': 'Lincoln',
    '18': 'Near West',
    '15': 'Austin',
    '31': None,
    '21': None
}


In [15]:
data_frame['street_num']=data_frame['Block'].str.split('X').str[0].str.replace('X','')

In [16]:
data_frame['street_name']=data_frame['street_num']+' '+data_frame['Block'].str.split(' ').str[1:].str.join(' ')

In [17]:
data_frame.head()

ID Case Number                    Date                      Block  \
0   5741943    HN549294  08/25/2007 09:22:18 AM         074XX N ROGERS AVE   
1     25953    JE240540  05/24/2021 03:06:00 PM        020XX N LARAMIE AVE   
2     26038    JE279849  06/26/2021 09:24:00 AM      062XX N MC CORMICK RD   
3  13279676    JG507211  11/09/2023 07:30:00 AM           019XX W BYRON ST   
4  13274752    JG501049  11/12/2023 07:59:00 AM  086XX S COTTAGE GROVE AVE   

   IUCR Primary Type                                        Description  \
0  0560      ASSAULT                                             SIMPLE   
1  0110     HOMICIDE                                FIRST DEGREE MURDER   
2  0110     HOMICIDE                                FIRST DEGREE MURDER   
3  0620     BURGLARY                                     UNLAWFUL ENTRY   
4  0454      BATTERY  AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...   

  Location Description  Arrest  Domestic  Beat  District  Ward  \
0                OTHER   False     False  2422      24.0  49.0   
1               STREET    True     False  2515      25.0  36.0   
2          PARKING LOT    True     False  1711      17.0  50.0   
3            APARTMENT   False     False  1922      19.0  47.0   
4   SMALL RETAIL STORE    True     False   632       6.0   6.0   

   Community Area FBI Code  X Coordinate  Y Coordinate  Year  \
0             1.0      08A           NaN           NaN  2007   
1            19.0      01A     1141387.0     1913179.0  2021   
2            13.0      01A     1152781.0     1941458.0  2021   
3             5.0       05     1162518.0     1925906.0  2023   
4            44.0      08B     1183071.0     1847869.0  2023   

               Updated On   Latitude  Longitude  \
0  08/17/2015 03:03:40 PM        NaN        NaN   
1  11/18/2023 03:39:49 PM  41.917838 -87.755969   
2  11/18/2023 03:39:49 PM  41.995219 -87.713355   
3  11/18/2023 03:39:49 PM  41.952345 -87.677975   
4  12/09/2023 03:41:24 PM  41.737751 -87.604856   

                        Location street_num              street_name  
0                            NaN        074         074 N ROGERS AVE  
1  (41.917838056, -87.755968972)        020        020 N LARAMIE AVE  
2  (41.995219444, -87.713354912)        062      062 N MC CORMICK RD  
3  (41.952345086, -87.677975059)        019           019 W BYRON ST  
4  (41.737750767, -87.604855911)        086  086 S COTTAGE GROVE AVE

We fill spatial values using the block name by removing 'X' in it. along with community area name, chicago, illinois

we fill comunity names using spatial values

In [18]:
ca_df=pd.DataFrame({'keys':community_areas.keys(),'values':community_areas.values()})
ca_df['keys']=ca_df['keys'].astype(float)
ca_names=dict(zip(ca_df['keys'],ca_df['values']))

In [19]:
data_frame['community area name']=data_frame['Community Area'].map(ca_names)

In [20]:
#filling location values

In [21]:
lca_df=data_frame[(data_frame['Location'].isnull())&(data_frame['Community Area'].isnull()==False)]

In [22]:
lca_df.head()

ID Case Number                    Date               Block  IUCR  \
0    5741943    HN549294  08/25/2007 09:22:18 AM  074XX N ROGERS AVE  0560   
27  12416972    JE293535  10/01/2020 12:01:00 AM     031XX W 53RD PL  1753   
28  12416974    JE293464  08/10/2011 12:01:00 AM     031XX W 53RD PL  1753   
29  12536164    JE439378  09/24/2015 12:00:00 AM     031XX W 53RD PL  1753   
30  12536166    JE439332  09/07/2014 12:00:00 AM     031XX W 53RD PL  1753   

                  Primary Type                               Description  \
0                      ASSAULT                                    SIMPLE   
27  OFFENSE INVOLVING CHILDREN  SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER   
28  OFFENSE INVOLVING CHILDREN  SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER   
29  OFFENSE INVOLVING CHILDREN  SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER   
30  OFFENSE INVOLVING CHILDREN  SEXUAL ASSAULT OF CHILD BY FAMILY MEMBER   

   Location Description  Arrest  Domestic  Beat  District  Ward  \
0                 OTHER   False     False  2422      24.0  49.0   
27            RESIDENCE   False      True   923       9.0  14.0   
28            RESIDENCE   False      True   923       9.0  14.0   
29            APARTMENT   False      True   923       9.0  14.0   
30            APARTMENT   False      True   923       9.0  14.0   

    Community Area FBI Code  X Coordinate  Y Coordinate  Year  \
0              1.0      08A           NaN           NaN  2007   
27            63.0       02           NaN           NaN  2020   
28            63.0       02           NaN           NaN  2011   
29            63.0       02           NaN           NaN  2015   
30            63.0       02           NaN           NaN  2014   

                Updated On  Latitude  Longitude Location street_num  \
0   08/17/2015 03:03:40 PM       NaN        NaN      NaN        074   
27  09/14/2023 03:41:59 PM       NaN        NaN      NaN        031   
28  09/14/2023 03:41:59 PM       NaN        NaN      NaN        031   
29  09/14/2023 03:41:59 PM       NaN        NaN      NaN        031   
30  09/14/2023 03:41:59 PM       NaN        NaN      NaN        031   

         street_name community area name  
0   074 N ROGERS AVE         Rogers Park  
27     031 W 53RD PL           Gage Park  
28     031 W 53RD PL           Gage Park  
29     031 W 53RD PL           Gage Park  
30     031 W 53RD PL           Gage Park

In [23]:
lca_df['address']=lca_df['street_name']+', '+lca_df['community area name']+', Chicago, Illinois'

In [24]:
uni_address=lca_df['address'].unique()

In [25]:
#loading the missing coordinates

In [28]:
with open('final_coor_values.json','r') as file:
    coor_dict=json.load(file)

In [29]:
coor_dict['051 N NORTHWEST HWY, Jefferson Park, Chicago, Illinois']=['41.84820', '-87.62898']

In [30]:
lca_df['coordinates']=lca_df['address'].map(coor_dict)

In [31]:
data_frame['Latitude'].iloc[lca_df.index]=lca_df['coordinates'].astype(str).str.split(',').str[0]
data_frame['Longitude'].iloc[lca_df.index]=lca_df['coordinates'].astype(str).str.split(',').str[1]

In [32]:
data_frame.head()

ID Case Number                    Date                      Block  \
0   5741943    HN549294  08/25/2007 09:22:18 AM         074XX N ROGERS AVE   
1     25953    JE240540  05/24/2021 03:06:00 PM        020XX N LARAMIE AVE   
2     26038    JE279849  06/26/2021 09:24:00 AM      062XX N MC CORMICK RD   
3  13279676    JG507211  11/09/2023 07:30:00 AM           019XX W BYRON ST   
4  13274752    JG501049  11/12/2023 07:59:00 AM  086XX S COTTAGE GROVE AVE   

   IUCR Primary Type                                        Description  \
0  0560      ASSAULT                                             SIMPLE   
1  0110     HOMICIDE                                FIRST DEGREE MURDER   
2  0110     HOMICIDE                                FIRST DEGREE MURDER   
3  0620     BURGLARY                                     UNLAWFUL ENTRY   
4  0454      BATTERY  AGGRAVATED P.O. - HANDS, FISTS, FEET, NO / MIN...   

  Location Description  Arrest  Domestic  Beat  District  Ward  \
0                OTHER   False     False  2422      24.0  49.0   
1               STREET    True     False  2515      25.0  36.0   
2          PARKING LOT    True     False  1711      17.0  50.0   
3            APARTMENT   False     False  1922      19.0  47.0   
4   SMALL RETAIL STORE    True     False   632       6.0   6.0   

   Community Area FBI Code  X Coordinate  Y Coordinate  Year  \
0             1.0      08A           NaN           NaN  2007   
1            19.0      01A     1141387.0     1913179.0  2021   
2            13.0      01A     1152781.0     1941458.0  2021   
3             5.0       05     1162518.0     1925906.0  2023   
4            44.0      08B     1183071.0     1847869.0  2023   

               Updated On     Latitude      Longitude  \
0  08/17/2015 03:03:40 PM  [42.0157992   -87.6757558]   
1  11/18/2023 03:39:49 PM    41.917838     -87.755969   
2  11/18/2023 03:39:49 PM    41.995219     -87.713355   
3  11/18/2023 03:39:49 PM    41.952345     -87.677975   
4  12/09/2023 03:41:24 PM    41.737751     -87.604856   

                        Location street_num              street_name  \
0                            NaN        074         074 N ROGERS AVE   
1  (41.917838056, -87.755968972)        020        020 N LARAMIE AVE   
2  (41.995219444, -87.713354912)        062      062 N MC CORMICK RD   
3  (41.952345086, -87.677975059)        019           019 W BYRON ST   
4  (41.737750767, -87.604855911)        086  086 S COTTAGE GROVE AVE   

  community area name  
0         Rogers Park  
1      Belmont Cragin  
2          North Park  
3        North Center  
4             Chatham

In [33]:
null_count_gen(data_frame)

Null count  Null percentage(%)   dtypes
ID                             0            0.000000    int64
Case Number                    0            0.000000   object
Date                           0            0.000000   object
Block                          0            0.000000   object
IUCR                           0            0.000000   object
Primary Type                   0            0.000000   object
Description                    0            0.000000   object
Location Description       12364            0.154904   object
Arrest                         0            0.000000     bool
Domestic                       0            0.000000     bool
Beat                           0            0.000000    int64
District                      47            0.000589  float64
Ward                      614854            7.703286  float64
Community Area            613479            7.686059  float64
FBI Code                       0            0.000000   object
X Coordinate               87847            1.100604  float64
Y Coordinate               87847            1.100604  float64
Year                           0            0.000000    int64
Updated On                     0            0.000000   object
Latitude                    9203            0.115301   object
Longitude                   9210            0.115389   object
Location                   87847            1.100604   object
street_num                     0            0.000000   object
street_name                    0            0.000000   object
community area name       613555            7.687011   object

In [34]:
lat_null_index=data_frame[(data_frame['Latitude'].isnull()==False)&(data_frame['Longitude'].isnull())].index

In [35]:
data_frame['Latitude'].iloc[lat_null_index]=None

In [36]:
null_count_gen(data_frame)

Null count  Null percentage(%)   dtypes
ID                             0            0.000000    int64
Case Number                    0            0.000000   object
Date                           0            0.000000   object
Block                          0            0.000000   object
IUCR                           0            0.000000   object
Primary Type                   0            0.000000   object
Description                    0            0.000000   object
Location Description       12364            0.154904   object
Arrest                         0            0.000000     bool
Domestic                       0            0.000000     bool
Beat                           0            0.000000    int64
District                      47            0.000589  float64
Ward                      614854            7.703286  float64
Community Area            613479            7.686059  float64
FBI Code                       0            0.000000   object
X Coordinate               87847            1.100604  float64
Y Coordinate               87847            1.100604  float64
Year                           0            0.000000    int64
Updated On                     0            0.000000   object
Latitude                    9210            0.115389   object
Longitude                   9210            0.115389   object
Location                   87847            1.100604   object
street_num                     0            0.000000   object
street_name                    0            0.000000   object
community area name       613555            7.687011   object

In [263]:
final_data=data_frame[data_frame['Latitude'].isnull()==False].dropna(axis=1)

In [264]:
null_count_gen(final_data)

Null count  Null percentage(%)  dtypes
ID                     0                 0.0   int64
Case Number            0                 0.0  object
Date                   0                 0.0  object
Block                  0                 0.0  object
IUCR                   0                 0.0  object
Primary Type           0                 0.0  object
Description            0                 0.0  object
Arrest                 0                 0.0    bool
Domestic               0                 0.0    bool
Beat                   0                 0.0   int64
FBI Code               0                 0.0  object
Year                   0                 0.0   int64
Updated On             0                 0.0  object
Latitude               0                 0.0  object
Longitude              0                 0.0  object
Location               0                 0.0  object
street_num             0                 0.0  object
street_name            0                 0.0  object
only_sn                0                 0.0  object
address                0                 0.0  object

In [265]:
final_data.to_csv('final_data.csv',index=False)